In [ ]:
# Instalando bibliotecas 
!pip install --upgrade tweepy
!pip install snscrape
!pip install --upgrade networkx

In [2]:
# Importando biblioteca de acesso a API do Twitter
import tweepy as tw
#tw.__version__   aaa sb
import pytz
import datetime
import pandas as pd
import snscrape
pd.set_option('display.max_colwidth', None)

In [3]:
# Autenticando na API do Twitter

# Dados de Autenticação
# Caso ainda não tenha acesso a API do Twitter acesse essa página:
# https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAKtckwEAAAAAYV8r7pZHTOzX%2F1ZH7dywUUELKxU%3DfuB1CZ6BYXp8S0Jjf2md95jIifYwCLrc875I0Zt9kp5cNOsd6S' # Bear token pego com a conta do bandeijao unirio

# Autenticando
client = tw.Client(bearer_token)

In [ ]:
# Configurando a Pesquisa
query = "Bolsonaro  lang:pt"
data_inicio = "2023-01-01 14:00:00"
data_fim = "2023-01-01 14:30:00"

def converteData(data):
    # Formato esperado de data "2000-1-1 12:00:00"
    local_time = pytz.timezone("America/Sao_Paulo")
    utc_data = local_time.localize(datetime.datetime.strptime(data, "%Y-%m-%d %H:%M:%S"), is_dst=None).astimezone(pytz.utc)
    dtformat = '%Y-%m-%dT%H:%M:%SZ'
    time = datetime.datetime.utcnow()
    return utc_data.strftime(dtformat)

In [ ]:
# Extraindo os Tweets

response_tweets = client.search_recent_tweets(query, max_results=100,start_time=converteData(data_inicio), end_time=converteData(data_fim),
                                       user_fields=['id'], expansions=['author_id'], tweet_fields=['created_at'])
print(response_tweets)

Response(data=[<Tweet id=1609602853194158081 text='RT @siteptbr: 🚨 AGORA: Alexandre de Moraes planeja prender Bolsonaro. https://t.co/7RmCrILJhn'>, <Tweet id=1609602852216881157 text='RT @twitos: "Verás que um filho teu não foge à luta!" Jair Bolsonaro, nunca foi filho da Pátria, sempre foi filho da p*ta! #BolsonaroFujao…'>, <Tweet id=1609602851822600194 text='RT @GayerGus: Mourão está prestes a trair o Bolsonaro hoje às 20:00. \nVai ser a terceira facada sendo que as duas últimas vieram dos própri…'>, <Tweet id=1609602851516342272 text='RT @taoquei1: O povo achando que nessa virada de ano teremos um pronunciamento histórico do Mourão, mas sinto informar, será a oportunidade…'>, <Tweet id=1609602850379661314 text='RT @BolsonaroSP: 🇺🇸Bolsonaro, eterno presidente, o verdadeiro presidente. O apoio de pessoas decentes, que acreditam que o trabalho honesto…'>, <Tweet id=1609602847787606018 text='RT @TradutordoBR: Será que a turma que incentivou geral a votar nulo vai ter coragem de criticar

In [ ]:

# Convertendo a saída em Dataframe para melhor visualização

df_tweets_extraidos = pd.DataFrame.from_dict(response_tweets.data)
df_tweets_extraidos.head(10)

In [ ]:
# Contagem dos Tweets disponiveis na query pesquisada
response_contagem = client.get_recent_tweets_count(query, start_time=converteData(data_inicio), end_time=converteData(data_fim), granularity="day")

print('Quantidade de Tweets:' + str(response_contagem.meta['total_tweet_count']))

Quantidade de Tweets:12445


In [ ]:
# Para obter todos os Tweets iremos utilizar o Token para a pagina seguinte

print(response_tweets.meta['next_token'])

b26v89c19zqg8o3fqk13d60n366uv8s70f1ghf52slg8t


In [ ]:
data = []

# Gravando os Tweets coletados na primeira pagina
for tweet in response_tweets.data:
        data.append([tweet.author_id, tweet.created_at, tweet.edit_history_tweet_ids, tweet.id, tweet.text])

# Iterando enquanto houver mais resultados disponiveis, ou seja, uma proxima pagina        
while ("next_token" in response_tweets.meta):
    response_tweets = client.search_recent_tweets(query, next_token=response_tweets.meta['next_token'], max_results=100,
                                           start_time=converteData(data_inicio), end_time=converteData(data_fim),user_fields=['id'], 
                                           expansions=['author_id'], tweet_fields=['created_at'])
    for tweet in response_tweets.data:
        data.append([tweet.author_id, tweet.created_at, tweet.edit_history_tweet_ids, tweet.id, tweet.text])

df_tweets = pd.DataFrame(data, columns = ['author_id','created_at','edit_history_tweet_ids','id','text'])


In [ ]:
df_tweets.head(10)

In [ ]:
# Nuvem de palavras
from wordcloud import  WordCloud, STOPWORDS, ImageColorGenerator
import re

# Removendo duplicatas em Tweets
df_tweets = df_tweets.drop_duplicates(subset=['text'])

# Atribundo todos os Tweets um uma só variável
tweets=df_tweets['text']
palavras_tweets=" ".join(s for s in tweets)

palavras_tweets


'BOLSONARO BEM VINDO AO GIGANTE RT @acababozo: ACABOU O GOVERNO BOLSONARO RT @MValente2022: @Paulogeneroso Eu ainda estou cheia de perguntas : \n\n1) Onde está o GEN Braga Neto? \n2) O PR viajou por 30 dias. Disse q… lula presidente, bolsonaro fora do Brasil, 2023 tá ótimo já, né RT @eutauat: Holiday está disposto a ocupar o vácuo deixado por Bolsonaro no quesito desinformação. Os artistas que participarão da posse d… RT @WesternTimes_: O presidente brasileiro Jair Bolsonaro já está na Flórida, em Orlando, e pode se reunir com o presidente Trump a qualque… @jairbolsonaro Bolsonaro você vai ser preso? @Paulogeneroso Bolsonaro já traiu os brasileiros ao permitir que ficassem nos quartéis e fugir na última hora. Tudo farinha do mesmo saco. RT @ideias_gp: Chegou a hora de dizer adeus a Bolsonaro. E fazer a oposição dura que Lula e o PT merecem https://t.co/9myzXTXDU3 RT @ItaloPetraglia: Ficaram 4 anos falando que o Bolsonaro daria golpe\n\nNem qnd os caras PEDIRAM golpe 2 meses na frente d

In [ ]:
    # Configurando as StopWords
stop_word_ptbr = set(STOPWORDS)
stop_word_ptbr.update(['a','as','e','o','os','eu', 'esse' ,'daí','essa','é', 'vc','um', 'tá', 'pq', 'né', 'to','meu','meus','minha','minhas','seu','sua','seus','suas','de','do','da','em','que','se','não','ele','ela','t','co','https','uma','para','na','RT', 'é', 'pra', 'todo', 'mas', 'como'])

In [ ]:
# Gerando a Nuvem de Palavras
nuvem_tweets = WordCloud(stopwords=stop_word_ptbr,background_color='white',
                         width=800,height=800).generate(palavras_tweets)

In [ ]:
# Exibindo a nuvem de Palavras
import matplotlib.pyplot as plt

figura, eixo = plt.subplots(figsize=(8,8))
plt.imshow(nuvem_tweets)

In [ ]:
# Configurando a Pesquisa
query = "Bolsonaro  lang:pt"
data_inicio = "2023-01-01 14:00:00"
data_fim = "2023-01-01 14:30:00"

In [ ]:
# Contagem dos Tweets disponiveis na query pesquisada
response_contagem = client.get_recent_tweets_count(query, start_time=converteData(data_inicio), end_time=converteData(data_fim), granularity="day")

print('Quantidade de Tweets:' + str(response_contagem.meta['total_tweet_count']))

Quantidade de Tweets:12445


In [ ]:
# Requisição da primeira pagina
response_tweets = client.search_recent_tweets(query, max_results=100,start_time=converteData(data_inicio), end_time=converteData(data_fim),
                                       user_fields=['id','username'], expansions=['author_id','entities.mentions.username'], tweet_fields=['created_at'])
#response_tweets

In [ ]:
import networkx as nx

grafo = nx.DiGraph()
data = []
usuarios = [] 


#Requisição da primeira pagina
response_tweets = client.search_recent_tweets(query, max_results=100,start_time=converteData(data_inicio), end_time=converteData(data_fim),
                                       user_fields=['id','username'], expansions=['author_id','entities.mentions.username'], tweet_fields=['created_at'])

#Gravando a ID e username dos autores dos Tweets
for user in response_tweets.includes['users']:
    users_dict = {}
    users_dict[0] = user['id']
    users_dict[1] = user['username']
    usuarios.append(users_dict)

# Gravando os Tweets coletados na primeira pagina
for tweet in response_tweets.data:
        data.append([tweet.author_id, tweet.created_at, tweet.edit_history_tweet_ids, tweet.id, tweet.text])
        if "entities" in tweet:
          for mencao in tweet.entities['mentions']:            
              # Buscando o user_name do autor do Tweet
              username_autor_tweet = list(filter(lambda usuario: usuario[0] == tweet.author_id, usuarios))[0][1]
              # Criando uma aresta do autor Tweet para o(s) usuário(s) citado(s)
              grafo.add_edge(username_autor_tweet,mencao['username'])

            
            
# Iterando enquanto houver mais resultados disponiveis, ou seja, uma proxima pagina        
while ("next_token" in response_tweets.meta):
    response_tweets = client.search_recent_tweets(query, next_token=response_tweets.meta['next_token'], max_results=100,
                                           start_time=converteData(data_inicio), end_time=converteData(data_fim),user_fields=['id','username'], 
                                           expansions=['author_id','entities.mentions.username'], tweet_fields=['created_at'])
    # Gravando a ID e username dos autores dos Tweets
    for user in response_tweets.includes['users']:
        users_dict = {}
        users_dict[0] = user['id']
        users_dict[1] = user['username']
        usuarios.append(users_dict)
    
    for tweet in response_tweets.data:
        data.append([tweet.author_id, tweet.created_at, tweet.edit_history_tweet_ids, tweet.id, tweet.text])
        if "entities" in tweet:
          for mencao in tweet.entities['mentions']:            
              # Gravando a ID e username dos autores dos Tweets
              username_autor_tweet = list(filter(lambda usuario: usuario[0] == tweet.author_id, usuarios))[0][1]
              # Criando uma aresta do autor Tweet para o(s) usuário(s) citado(s)
              grafo.add_edge(username_autor_tweet,mencao['username'])

            
# Exportando o grafo para um arquivo            
nx.write_graphml_lxml(grafo, "grafo_extracao.graphml")        
 

df_tweets = pd.DataFrame(data, columns = ['author_id','created_at','edit_history_tweet_ids','id','text'])

In [ ]:
df_tweets.head(10)